# R2 Overview:

Remember to do:

Setup

List functions, then print code for one

In [2]:
%pip install r2pipe

  Using cached r2pipe-1.6.2.tar.gz (10 kB)
  Created wheel for r2pipe: filename=r2pipe-1.6.2-py3-none-any.whl size=11425 sha256=2200c4cca56119981a660c687ac36dadb1c0ad7c5c5951d7e67774086fa142e1
  Stored in directory: /home/parallels/.cache/pip/wheels/6f/b8/69/be0a45b6727965affea5a36445a16c8eb06622d1e088ca192b
Successfully built r2pipe
Note: you may need to restart the kernel to use updated packages.


In [19]:
import r2pipe

rp = r2pipe.open('if1')

In [20]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)
@register_line_magic
def r(line):
    print(line)
    print()
    print(rp.cmd(line))
    return

In [21]:
%r aa;e asm.syntax=att;afl;

aa;e asm.syntax=att;afl;

0x000004f0    1 42           entry0
0x00000520    4 50   -> 40   sym.deregister_tm_clones
0x00000560    4 66   -> 57   sym.register_tm_clones
0x000005b0    5 58   -> 51   sym.__do_global_dtors_aux
0x000004e0    1 6            sym.imp.__cxa_finalize
0x000005f0    1 10           entry.init0
0x000006a0    1 2            sym.__libc_csu_fini
0x000006a4    1 9            sym._fini
0x00000630    4 101          sym.__libc_csu_init
0x000005fa    4 43           main
0x000004b8    3 23           sym._init
0x00000000    5 97   -> 121  loc.imp._ITM_deregisterTMCloneTable



In [22]:
%r pdf @main

pdf @main

            ; DATA XREF from entry0 @ 0x50d
┌ 43: int main (int argc, char **argv, char **envp);
│           ; var int64_t var_8h @ rbp-0x8
│           ; var int64_t var_4h @ rbp-0x4
│           0x000005fa      55             pushq %rbp
│           0x000005fb      4889e5         movq %rsp, %rbp
│           0x000005fe      c745f8030000.  movl $3, var_8h
│           0x00000605      c745fc040000.  movl $4, var_4h
│           0x0000060c      8b45f8         movl var_8h, %eax
│           0x0000060f      3b45fc         cmpl var_4h, %eax
│       ┌─< 0x00000612      7d06           jge 0x61a
│       │   0x00000614      8345f805       addl $5, var_8h
│      ┌──< 0x00000618      eb04           jmp 0x61e
│      │└─> 0x0000061a      8345fc03       addl $3, var_4h
│      │    ; CODE XREF from main @ 0x618
│      └──> 0x0000061e      b800000000     movl $0, %eax
│           0x00000623      5d             popq %rbp
└           0x00000624      c3             retq



# Other Notes:

# R2: Set Break Points:


In [10]:
%r db 0x00000612
%r db 0x00000618

In [23]:
%r pdf @main

pdf @main

            ; DATA XREF from entry0 @ 0x50d
┌ 43: int main (int argc, char **argv, char **envp);
│           ; var int64_t var_8h @ rbp-0x8
│           ; var int64_t var_4h @ rbp-0x4
│           0x000005fa      55             pushq %rbp
│           0x000005fb      4889e5         movq %rsp, %rbp
│           0x000005fe      c745f8030000.  movl $3, var_8h
│           0x00000605      c745fc040000.  movl $4, var_4h
│           0x0000060c      8b45f8         movl var_8h, %eax
│           0x0000060f      3b45fc         cmpl var_4h, %eax
│       ┌─< 0x00000612      7d06           jge 0x61a
│       │   0x00000614      8345f805       addl $5, var_8h
│      ┌──< 0x00000618      eb04           jmp 0x61e
│      │└─> 0x0000061a      8345fc03       addl $3, var_4h
│      │    ; CODE XREF from main @ 0x618
│      └──> 0x0000061e      b800000000     movl $0, %eax
│           0x00000623      5d             popq %rbp
└           0x00000624      c3             retq



In [12]:
%r dc

Cannot continue, run ood?


In [13]:
%r dr

rax = 0x00000000
rbx = 0x00000000
rcx = 0x00000000
rdx = 0x00000000
rsi = 0x00000000
rdi = 0x00000000
r8 = 0x00000000
r9 = 0x00000000
r10 = 0x00000000
r11 = 0x00000000
r12 = 0x00000000
r13 = 0x00000000
r14 = 0x00000000
r15 = 0x00000000
rip = 0x00000000
rbp = 0x00000000
rflags = 0x00000000
rsp = 0x00000000



In [ ]:
%r dc
%r px @rbp-0x4
%r ds